In [1]:
import pandas as pd
import tensorflow as tf
import keras as keras

import re
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

%matplotlib inline

/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/amit/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/amit/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

In [2]:
train_df = pd.read_csv("../../data/tweet/train.csv")
test_df = pd.read_csv("../../data/tweet/test.csv")

In [3]:
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
combi_df = train_df.append(test_df, ignore_index=True, sort=False)

In [161]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

# remove twitter handles (@user)
combi_df['tidy_tweet'] = np.vectorize(remove_pattern)(combi_df['tweet'], "@[\w]*")

combi_df['tidy_tweet'] = combi_df['tidy_tweet'].str.replace("[^a-zA-Z]", " ")

combi_df['tidy_tweet'] = combi_df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [162]:
combi_df.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks lyft credit cause they offer wheelchair...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,model love take with time
4,5,0.0,factsguide: society now #motivation,factsguide society motivation


In [163]:
tokenized_tweet = combi_df['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, lyft, credit, cause, they, offer, whe...
2                              [bihday, your, majesty]
3                      [model, love, take, with, time]
4                    [factsguide, society, motivation]
Name: tidy_tweet, dtype: object

In [164]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [when, father, dysfunct, selfish, drag, kid, i...
1    [thank, lyft, credit, caus, they, offer, wheel...
2                              [bihday, your, majesti]
3                      [model, love, take, with, time]
4                          [factsguid, societi, motiv]
Name: tidy_tweet, dtype: object

In [165]:
for i in range(len(tokenized_tweet)):
  tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi_df['tidy_tweet'] = tokenized_tweet

In [166]:
combi_df.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thank lyft credit caus they offer wheelchair v...
2,3,0.0,bihday your majesty,bihday your majesti
3,4,0.0,#model i love u take with u all the time in ...,model love take with time
4,5,0.0,factsguide: society now #motivation,factsguid societi motiv


In [167]:
max_in_len = 60
MAX_VOCAB_SIZE=15000
# texts_to_sequences
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
t = Tokenizer(MAX_VOCAB_SIZE)
t.fit_on_texts(combi_df['tidy_tweet'])
# print(t.word_counts)
word_index = t.word_index
print("word_index : %d" % len(word_index))
combi_X = t.texts_to_sequences(combi_df['tidy_tweet'])
combi_X = pad_sequences(combi_X, maxlen=max_in_len, padding="pre")

word_index : 38973


In [168]:
train_df = combi_df[:31962]
test_df = combi_df[31962:]

In [169]:
train_X = combi_X[:31962]
test_X = combi_X[31962:]

In [170]:
train_df.shape[0]

31962

In [171]:
train_Y = train_df["label"]
train_Y.size

31962

In [187]:
# vocab_size = 10000
# vocab_size = len(word_index) + 1
embed_size = 150

hidden_size = 200

from keras.models import *
from keras.layers import *
from keras.optimizers import *
model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, embed_size, input_length=max_in_len))
# model.add(Dense(64, activation="relu"))
# model.add(Dense(1, activation="relu"))
# model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(hidden_size, dropout=0.1))
model.add(Dropout(0.5))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 60, 150)           2250000   
_________________________________________________________________
dropout_29 (Dropout)         (None, 60, 150)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 200)               280800    
_________________________________________________________________
dropout_30 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                6432      
_________________________________________________________________
dropout_31 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 33        
Total para

In [188]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [189]:
# opt = SGD(lr=0.0001)
model.compile(optimizer="nadam",
              loss='binary_crossentropy', metrics=[f1])

In [190]:
model.fit(train_X, train_Y, epochs=3, validation_split=0.05)

Train on 30363 samples, validate on 1599 samples
Epoch 1/3
30363/30363 [==============================] - 81s 3ms/step - loss: 0.1856 - f1: 0.3392 - val_loss: 0.1193 - val_f1: 0.6237
Epoch 2/3
30363/30363 [==============================] - 79s 3ms/step - loss: 0.0863 - f1: 0.6601 - val_loss: 0.1406 - val_f1: 0.6264
Epoch 3/3
30363/30363 [==============================] - 79s 3ms/step - loss: 0.0545 - f1: 0.7543 - val_loss: 0.1405 - val_f1: 0.6780


In [192]:
model.fit(train_X, train_Y, epochs=1, validation_split=0.05)

Train on 30363 samples, validate on 1599 samples
Epoch 1/1
30363/30363 [==============================] - 72s 2ms/step - loss: 0.0223 - f1: 0.8462 - val_loss: 0.2202 - val_f1: 0.6945


In [193]:
y_pred = model.predict(train_X)

In [194]:
y_pred.size

31962

In [195]:
y_pred

array([[5.0975038e-09],
       [1.3382709e-06],
       [2.9838024e-08],
       ...,
       [1.2795651e-09],
       [9.9999571e-01],
       [3.1821737e-03]], dtype=float32)

In [196]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print('Confusion Matrix')
print(confusion_matrix(train_df["label"], (y_pred > 0.5).astype(np.int)))

Confusion Matrix
[[29688    32]
 [  132  2110]]


In [197]:
test_df = combi_df.loc[31962:]

# t = Tokenizer()
# t.fit_on_texts(test_df['tidy_tweet'])
# print(t.word_counts)
# test_X = t.texts_to_sequences(test_df['tidy_tweet'])
# test_X = pad_sequences(test_X, maxlen=max_in_len, padding="pre")

test_y = model.predict(test_X)

In [198]:
final_y = (test_y > 0.5).astype(np.int)

In [199]:
# final_y[100:400]

In [200]:
# test_df['label'] = test_y.round()
test_df['label'] = final_y
# test_df["label"] = test_df["label"].apply(lambda x: '1' if x > 0.3 else '0')
test_df.head()

/home/amit/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,label,tweet,tidy_tweet
31962,31963,0,#studiolife #aislife #requires #passion #dedic...,studiolif aislif requir passion dedic willpow ...
31963,31964,1,@user #white #supremacists want everyone to s...,white supremacist want everyon bird movi here
31964,31965,0,safe ways to heal your #acne!! #altwaystohe...,safe way heal your acn altwaystoh healthi heal
31965,31966,0,is the hp and the cursed child book up for res...,curs child book reserv alreadi where when harr...
31966,31967,0,"3rd #bihday to my amazing, hilarious #nephew...",bihday amaz hilari nephew ahmir uncl dave love...


In [201]:
test_df.head()

,id,label,tweet,tidy_tweet
31962,31963,0,#studiolife #aislife #requires #passion #dedic...,studiolif aislif requir passion dedic willpow ...
31963,31964,1,@user #white #supremacists want everyone to s...,white supremacist want everyon bird movi here
31964,31965,0,safe ways to heal your #acne!! #altwaystohe...,safe way heal your acn altwaystoh healthi heal
31965,31966,0,is the hp and the cursed child book up for res...,curs child book reserv alreadi where when harr...
31966,31967,0,"3rd #bihday to my amazing, hilarious #nephew...",bihday amaz hilari nephew ahmir uncl dave love...


In [202]:
submission = test_df[['id','label']]
submission.to_csv('rnn1.csv', index=False) # writing data to a CSV file